# Lab 3 - The Matrix Form and The Duality Theory

<b>Information on group members:</b><br>
1) Student ID, Name and last name <br>
2) Student ID, Name and last name

In [1]:
from pulp import *  
import numpy as np
import pandas as pd
import itertools
from IPython.display import display

# 1) The Matrix Form - Fundamental Insight (finish this part to get 3.0)

1.1) Given is the below (primal) linear problem:

primal problem:<br>
MAXIMIZE<br>
4*x1 + 2*x2 + 3*x3 <br>

SUBJECT TO<br>
1_constraint: x1 + x2 <= 10<br>
2_constraint: 2*x2 + x3 <= 12<br>
3_constraint: 3*x1 + 2*x3 <= 15<br>
4_constraint: x1 + x2 + x3 <= 20<br>

VARIABLES<br>
x1 Continuous<br>
x2 Continuous<br>
x3 Continuous<br>

1.2) Use the PuLP library to solve the above problem. Identify the optimal solution: the values for basic variables and the corresponding value for the objective function.

In [2]:
# Create a linear programming problem
model = LpProblem(name="PrimalProblem", sense=LpMaximize)

# Define the variables
x1 = LpVariable(name="x1", lowBound=0)
x2 = LpVariable(name="x2", lowBound=0)
x3 = LpVariable(name="x3", lowBound=0)

# Define the objective function
obj_func = 4 * x1 + 2 * x2 + 3 * x3
model += (obj_func, "objective_function")

# Add constraints
model += (x1 + x2 <= 10, "1_constraint")
model += (2 * x2 + x3 <= 12, "2_constraint")
model += (3 * x1 + 2 * x3 <= 15, "3_constraint")
model += (x1 + x2 + x3 <= 20, "4_constraint")

model

PrimalProblem:
MAXIMIZE
4*x1 + 2*x2 + 3*x3 + 0
SUBJECT TO
1_constraint: x1 + x2 <= 10

2_constraint: 2 x2 + x3 <= 12

3_constraint: 3 x1 + 2 x3 <= 15

4_constraint: x1 + x2 + x3 <= 20

VARIABLES
x1 Continuous
x2 Continuous
x3 Continuous

In [3]:
status = model.solve()

# Print status
print(f"status: {model.status}, {LpStatus[model.status]}")

# Print objective value
print(f"objective: {model.objective.value()}")

model.variables()
print(x1.value())
print(x2.value())
print(x3.value())

# Print variable values
for var in model.variables():
    print(f"{var.name}: {var.value()}")

status: 1, Optimal
objective: 31.428571379999998
4.4285714
5.5714286
0.85714286
x1: 4.4285714
x2: 5.5714286
x3: 0.85714286


1.3) In this exercise, you are asked to identify ALL basic (feasible and not) solutions to the above problem. We will do this naively. Specifically, you are asked to use the fundamental insight to build a final simplex tableau for each possible base. Therefore, you need first to initialize the data: c, b, A matrixes, and it is suggested to initialize the auxiliary matrix M defined as M = A + (concatenate) I (identity matrix). Note that the problem should be formulated in the augmented form. Then, you have to iterate over each possible base B, compute B-1, and other relevant parts for the simplex tableau.<br><br>
a) Identify the optimal solution using the optimality condition; print it (Z value and values for basic variables); compare thus derived solution with the optimum found using the PuLP library (obviously, both solutions should be the same). <br>
b) Count the number of feasible and infeasible solutions. How many (all) basic solutions to the problem can be identified? <br><br>
It is suggested to use the NumPy library for performing matrix operations. 

In [4]:
# Define the matrices and vectors
c = np.array([4, 2, 3, 0, 0, 0, 0])
b = np.array([10, 12, 15, 20])
A = np.array([[1, 1, 0],
              [0, 2, 1],
              [3, 0, 2],
              [1, 1, 1]])

In [5]:
import numpy as np

def simplex_components(c, b, A, base):
    # Calculate the basis matrix
    B = A[:, base]
    # Calculate the inverse of the basis matrix
    try:
        B_inv = np.linalg.inv(B)
    except np.linalg.LinAlgError:
        return None

    # Calculate the right-hand side values
    b_prime = B_inv.dot(b)

    # Calculate the coefficients for the decision variables in the basis
    A_prime = B_inv.dot(A)

    # Calculate the row 0 coefficients for the non-basic variables (decision variables)
    # and the objective function value
    c_B = c[base]
    row_0_decision = c - np.dot(c_B, A_prime)

    # The objective function value is the dot product of the inverse of the basis
    # and the right-hand side values
    z = np.dot(c_B, b_prime)

    slack = c_B @ B_inv
    decision_var = c - slack @ A
    decision_var = decision_var[:3]

    return B_inv, A_prime, b_prime, z, row_0_decision, decision_var, slack

# Define the input parameters
c = np.array([4, 2, 3, 0, 0, 0, 0])
b = np.array([10, 12, 15, 20])
A = np.array([
    [1, 1, 0, 1, 0, 0, 0],
    [0, 2, 1, 0, 1, 0, 0],
    [3, 0, 2, 0, 0, 1, 0],
    [1, 1, 1, 0, 0, 0, 1]
])

# Check all combinations of bases
feasible_solution_count = 0
for base in itertools.combinations(range(A.shape[1]), A.shape[0]):
    components = simplex_components(c, b, A, list(base))
    if components is not None:
        B_inv, A_prime, b_prime, z, row_0_decision, decision_var, slack = components
        if np.all(b_prime >= 0):
            feasible_solution_count += 1
            if np.all(slack >= 0) and np.all(decision_var >= 0):
                print(f'optimal alert {base} : {z}')

print(f"Number of feasible solutions: {feasible_solution_count}")
    

optimal alert (0, 1, 2, 6) : 31.428571428571423
optimal alert (0, 1, 4, 6) : 30.0
optimal alert (0, 3, 4, 6) : 20.0
optimal alert (1, 2, 3, 6) : 27.0
optimal alert (1, 3, 5, 6) : 12.0
optimal alert (3, 4, 5, 6) : 0.0
Number of feasible solutions: 8


## 2) The Duality Theory (finish this part + part 1 + to get 5.0)

2.1) Model the dual problem to the above solved primal one, using the PuLP library. Then, solve it and compare the derived optimum with the optimum for the primal problem. Are they equal?

In [6]:
model = LpProblem(name="DualProblem", sense=LpMinimize)

# Define the variables
y1 = LpVariable(name="y1", lowBound=0)
y2 = LpVariable(name="y2", lowBound=0)
y3 = LpVariable(name="y3", lowBound=0)
y4 = LpVariable(name="y4", lowBound=0)

# Define the objective function
obj_func = 10 * y1 + 12 * y2 + 15 * y3 + 20 * y4
model += (obj_func, "objective_function")

# Add constraints
model += (y1 + 3 * y3 + y4 >= 4, "1_constraint")
model += (y1 + 2 * y2 + y4 >= 2, "2_constraint")
model += (y2 + 2 * y3 + y4 >= 3, "3_constraint")

model

DualProblem:
MINIMIZE
10*y1 + 12*y2 + 15*y3 + 20*y4 + 0
SUBJECT TO
1_constraint: y1 + 3 y3 + y4 >= 4

2_constraint: y1 + 2 y2 + y4 >= 2

3_constraint: y2 + 2 y3 + y4 >= 3

VARIABLES
y1 Continuous
y2 Continuous
y3 Continuous
y4 Continuous

In [7]:
### Solve here:
status = model.solve()

# Print status
print(f"status: {model.status}, {LpStatus[model.status]}")

# Print objective value
print(f"objective: {model.objective.value()}")
print(y1.value())
print(y2.value())
print(y3.value())
print(y4.value())



status: 1, Optimal
objective: 31.42857072
0.57142857
0.71428571
1.1428571
0.0


2.2) This exercise is based on the exercise 1.3 (copy & paste solution). Here, you are asked to iterate over all basic solutions (as in 1.3) and store them along with their complementary dual solutions. Solutions should be stored in the PRIMAL_DUAL_SOLUTIONS list and sorted according to the objective value Z = W. Analyze their optimality and feasibility. Finally, you have to display all basic solutions wlong with their complementary solutions (you can use the provided piece of code written using the pandas library). <br><br>

PRIMAL_DUAL_SOLUTIONS is defined as a table consisting of n rows, where n is the number of basic solutions to the problem, and 21 columns. The columns are defined as follows:<br>
Col. 1: The objective value Z<br>
Col. 2-4: The values for decision variables (primal solution)<br>
Col. 5-8: The values for slack variables (primal solution)<br>
Col. 9: P_F = Y or N, Y/N = primal solution is feasible/infeasible<br>
Col. 10: P_O = Y or N, Y/N = primal solution is optimal/is not optimal<br>
Col. 11: P_STATE = -/suboptimal/superoptimal/optimal; depends on P_F and P_O (primal)<br>
Col. 12: D_STATE = -/suboptimal/superoptimal/optimal; depends on D_F and D_O (dual)<br>
Col. 13: D_F = Y or N, Y/N = dual solution is feasible/infeasible<br>
Col. 14: D_O = Y or N, Y/N = dual solution is optimal/is not optimal<br>
Col. 15-18: The values for decision variables (dual solution)<br>
Col. 19-21: The values for surplus variables (dual solution)<br><br>

Reminder: sort solutions according to Z; analyze how their states change with the increase of Z.

In [8]:
for x_b, is_feasible, basis_indices in solutions:
    
    print(f"x_b = {x_b}, basis_indices = {basis_indices}")

NameError: name 'solutions' is not defined